# 自定义 C++/CUDA 算子


> 本教程也适用于 AMD ROCm，无需额外修改。

PyTorch 提供了一个大型的可用于张量的操作符库（例如 `torch.add`、`torch.sum` 等）。但是，您可能希望将新的自定义操作符引入 PyTorch。本教程演示了编写用 C++/CUDA 的自定义算子的幸运途径。

对于我们的教程，我们将演示如何编写与 PyTorch 子系统组合的 fused 乘法加法的 C++ 和 CUDA 操作符。操作的语义如下：

```python
def mymuladd(a: Tensor, b: Tensor, c: float):
    return a * b + c
```

您可以在此处找到本教程的端到端工作[示例](https://github.com/pytorch/extension-cpp)。

## 设置构建系统

如果您正在开发自定义 C++/CUDA 代码，则必须对其进行编译。请注意，如果您正在与已经绑定到预编译 C++/CUDA 代码的 Python 库交互，则可以考虑编写自定义 Python 算子（[自定义 Python 算子](https://pytorch.org/tutorials/advanced/python_custom_ops.html#python-custom-ops-tutorial)）。

使用 [`torch.utils.cpp_extension`](https://pytorch.org/docs/stable/cpp_extension.html) 编译自定义 C++/CUDA 代码以用于 PyTorch C++ 扩展，可以使用 `setuptools`“Ahead of Time(AoT)” 构建，也可以通过 [load_inline](https://pytorch.org/docs/stable/cpp_extension.html#torch.utils.cpp_extension.load_inline)“Just in Time(JIT)”构建；我们将重点介绍 AoT 风格。

使用 `cpp_extension` 非常简单，只需编写以下 `setup.py`：

In [ ]:
from setuptools import setup, Extension
from torch.utils import cpp_extension

setup(
    name="extension_cpp",
    ext_modules=[
        cpp_extension.CppExtension(
            "extension_cpp",
            ["muladd.cpp"],
            extra_compile_args={"cxx":["-DPy_LIMITED_API=0x0309000"]},
            py_limited_api=True
        )
    ],
    cmdclass={'build_ext': cpp_extension.BuildExtension},
    options={'bdist_whell': {"py_limited_api": "cp39"}}
)

如果您需要编译 CUDA 代码（例如 .cu 文件），则改用 [`torch.utils.cpp_extension.CUDAExtension`](https://pytorch.org/docs/stable/cpp_extension.html#torch.utils.cpp_extension.CUDAExtension)。请参阅[ extension-cpp](https://github.com/pytorch/extension-cpp) 以了解如何设置的示例。

上述示例代表了我们所说的 CPython 不可知轮子(agnostic wheel)，这意味着我们正在构建一个可以在多个 CPython 版本上运行的单个轮子（类似于纯 Python 包）。CPython 不可知性对于最大限度地减少自定义库需要支持和发布的 wheel 数量是可取的。我们希望支持的最低版本是 3.9，因为它是目前支持的最旧版本，因此我们在整个设置代码中使用相应的十六进制代码和说明符。我们建议在与您想要支持的最低 CPython 版本相同的环境中构建扩展，以最大限度地减少未知行为，因此，在这里，我们在 CPython 3.9 环境中构建扩展。构建后，这个单个 wheel 将可在任何 CPython 环境 3.9+ 中运行。要实现这一点，有三行关键代码需要注意。

首先是将 `extra_compile_args` 中的 `Py_LIMITED_API` 指定为您想要支持的最低 CPython 版本：

In [ ]:
extra_compile_args={"cxx": ["-DPy_LIMITED_API=0x03090000"]},

定义 `Py_LIMITED_API` 标志有助于验证扩展是否实际上仅使用 [CPython 稳定有限 API](https://docs.python.org/3/c-api/stable.html)，这是构建 CPython 不可知轮盘的必要条件。如果不满足此要求，则可能会构建一个看起来与 CPython 无关但会在另一个 CPython 环境中崩溃或更糟的是默默错误的 Wheel。注意避免使用不稳定的 CPython API，例如来自 `libtorch_python` 的 API（特别是 `pytorch/python` 绑定），并且仅使用来自 libtorch 的 API（ATen 对象、算子和调度程序）。我们强烈建议定义 `Py_LIMITED_API` 标志，以帮助确定扩展是否符合要求并且作为 CPython 不可知 Wheel 是安全的。请注意，定义此标志并不能完全保证构建的 Wheel 与 CPython 无关，但总比狂野西部好。[Python 文档](https://docs.python.org/3/c-api/stable.html#limited-api-caveats)中提到了几个注意事项，您应该自己测试并验证 Wheel 是否真正与相关的 CPython 版本无关。

第二行和第三行指定了 `py_limited_api`，告知 `setuptools` 您打算构建一个与 CPython 无关的 wheel，并将相应地影响 wheel 的命名：

```python
setup(name="extension_cpp",
      ext_modules=[
          cpp_extension.CppExtension(
            ...,
            py_limited_api=True)],  # 跨多个 Python 版本构建 1 个 Wheel
      ...,
      options={"bdist_wheel": {"py_limited_api": "cp39"}}  # 3.9 是最低支持的 Python 版本
)
```

必须将 `py_limited_api=True` 指定为 `CppExtension/CUDAExtension` 的参数，以及“`bdist_wheel`”命令的选项，并使用最低支持的 CPython 版本（在本例中为 3.9）。因此，本教程中的设置将构建一个正确命名的 wheel，该 wheel 可以安装在多个 CPython 版本 >=3.9 上。

如果您的扩展使用稳定​​有限集之外的 CPython API，那么您无法构建 CPython 无关 wheel！您应该为每个 CPython 版本构建一个 wheel，如下所示：

In [ ]:
from setuptools import setup, Extension
from torch.utils import cpp_extension

setup(name="extension_cpp",
      ext_modules=[
          cpp_extension.CppExtension(
            "extension_cpp",
            ["muladd.cpp"])],
      cmdclass={'build_ext': cpp_extension.BuildExtension},
)

## 定义自定义操作并添加后端实现

首先，让我们编写一个计算 `mymuladd` 的 C++ 函数：

```c++
at::Tensor mymuladd_cpu(at::Tensor a, const at::Tensor& b, double c) {
  TORCH_CHECK(a.sizes() == b.sizes());
  TORCH_CHECK(a.dtype() == at::kFloat);
  TORCH_CHECK(b.dtype() == at::kFloat);
  TORCH_INTERNAL_ASSERT(a.device().type() == at::DeviceType::CPU);
  TORCH_INTERNAL_ASSERT(b.device().type() == at::DeviceType::CPU);
  at::Tensor a_contig = a.contiguous();
  at::Tensor b_contig = b.contiguous();
  at::Tensor result = torch::empty(a_contig.sizes(), a_contig.options());
  const float* a_ptr = a_contig.data_ptr<float>();
  const float* b_ptr = b_contig.data_ptr<float>();
  float* result_ptr = result.data_ptr<float>();
  for (int64_t i = 0; i < result.numel(); i++) {
    result_ptr[i] = a_ptr[i] * b_ptr[i] + c;
  }
  return result;
}
```

为了从 PyTorch 的 Python 前端使用它，我们需要使用 `TORCH_LIBRARY` API 将其**注册**为 PyTorch 算子。这自动将算子绑定到 Python。

算子注册是一个两步过程：
+ 定义算子 - 此步骤确保 PyTorch 知道新算子
+ 注册后端实现 - 在此步骤中，各种后端（例如 CPU 和 CUDA）的实现与算子相关联

### 定义算子

要定义算子，请按照以下步骤操作：

+ 为算子选择一个命名空间。我们**建议将命名空间作为顶级项目的名称**；我们将在本教程中使用 `extension_cpp`
+ 提供一个**模式字符串**，指定算子的输入/输出类型以及输入张量是否会发生变化。除了张量和浮点数之外，我们还支持更多类型；有关更多详细信息，请参阅[自定义算子手册](https://pytorch.org/docs/main/notes/custom_operators.html)
+ 如果您正在编写可以改变其输入张量的算子，请参阅此处（[创建可变算子](https://pytorch.org/tutorials/advanced/cpp_custom_ops.html#mutable-ops)）了解如何指定该算子

```c++
TORCH_LIBRARY(extension_cpp, m) {
    // 请注意，模式中的 float 对应于 C++ double 类型
    // 和 Python float 类型。
    m.def("mymuladd(Tensor a, Tensor b, float c)->Tensor");
}
```

这使得该算子可通过 Python 通过 `torch.ops.extension_cpp.mymuladd` 获得。

### 为算子注册后端实现

使用 `TORCH_LIBRARY_IMPL` 为算子注册后端实现。

```c++
TORCH_LIBRARY_IMPL(extension_cpp, CPU, m) {
    m.impl("mymuladd", &mymuladd_cpu);
}
```

如果您还有 `myaddmul` 的 CUDA 实现，则可以在单独的 `TORCH_LIBRARY_IMPL` 块中注册它：

```c++
__global__ void muladd_kernel(int numel, const float* a, const float* b, float c, float* result) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < numel) result[idx] = a[idx] * b[idx] + c;
}

at::Tensor mymuladd_cuda(const at::Tensor& a, const at::Tensor& b, double c) {
  TORCH_CHECK(a.sizes() == b.sizes());
  TORCH_CHECK(a.dtype() == at::kFloat);
  TORCH_CHECK(b.dtype() == at::kFloat);
  TORCH_INTERNAL_ASSERT(a.device().type() == at::DeviceType::CUDA);
  TORCH_INTERNAL_ASSERT(b.device().type() == at::DeviceType::CUDA);
  at::Tensor a_contig = a.contiguous();
  at::Tensor b_contig = b.contiguous();
  at::Tensor result = torch::empty(a_contig.sizes(), a_contig.options());
  const float* a_ptr = a_contig.data_ptr<float>();
  const float* b_ptr = b_contig.data_ptr<float>();
  float* result_ptr = result.data_ptr<float>();

  int numel = a_contig.numel();
  muladd_kernel<<<(numel+255)/256, 256>>>(numel, a_ptr, b_ptr, c, result_ptr);
  return result;
}

TORCH_LIBRARY_IMPL(extension_cpp, CUDA, m) {
  m.impl("mymuladd", &mymuladd_cuda);
}
```

### 为一个算子添加 `torch.compile` 支持

要为算子添加 `torch.compile` 支持，我们必须添加 `FakeTensor` 内核（也称为“元内核(meta kernel)”或“抽象实现(abstract impl)”）。`FakeTensor` 是具有元数据（例如形状、数据类型、设备）但没有数据的张量：算子的 FakeTensor 内核指定如何根据输入张量的元数据计算输出张量的元数据。FakeTensor 内核应返回您选择的具有正确张量元数据（形状/步幅/数据类型/设备）的虚拟张量。

我们建议通过 `torch.library.register_fake` API 从 Python 完成此操作，但也可以从 C++ 完成此操作（有关更多详细信息，请参阅[自定义算子手册](https://pytorch.org/docs/main/notes/custom_operators.html)）。

```python
# 重要提示：在调用添加 C++ 自定义算子注册的 ``torch.library`` API 之前，
# 应首先加载 C++ 自定义算子定义。以下导入将加载我们的
# C++ 自定义算子定义。
# 请注意，如果您追求 Python 不可知论(agnosticism)，则应改用
# ``load_library(...)`` API 调用。有关
# 更多详细信息，请参阅下一节。
from . import _C

@torch.library.register_fake("extension_cpp::mymuladd")
def _(a, b, c):
    torch._check(a.shape == b.shape)
    torch._check(a.dtype == torch.float)
    torch._check(b.dtype == torch.float)
    torch._check(a.device == b.device)
    return torch.empty_like(a)
```

### 设置混合 Python/C++ 注册

在本教程中，我们在 C++ 中定义了一个自定义算子，在 C++ 中添加了 CPU/CUDA 实现，并在 Python 中添加了 FakeTensor 内核和反向公式。加载（或导入）这些注册的顺序很重要（以错误的顺序导入会导致错误）。

要将自定义算子与混合 Python/C++ 注册一起使用，我们必须首先加载包含自定义算子定义的 C++ 库，然后调用 `torch.library` 注册 API。这可以通过三种方式实现：

1. 加载包含自定义算子定义的 C++ 库的第一种方法是为 `_C` 定义一个虚拟 Python 模块。然后，在 Python 中，当您使用 `import _C` 导入模块时，将加载与扩展相对应的 `.so` 文件，并运行 `TORCH_LIBRARY` 和 `TORCH_LIBRARY_IMPL` **静态初始化程序**。可以使用 `PYBIND11_MODULE` 创建一个虚拟 Python 模块（如下所示），但您会注意到，这不会使用 `Py_LIMITED_API` 进行编译，因为 pybind11 不承诺仅使用稳定有限的 CPython API！使用下面的代码，您很遗憾无法为您的扩展构建一个与 CPython 无关的 Wheel！（预示：我想知道第二种方法是什么 ;）。

```c++
// 例如，在 not_agnostic/csrc/extension_BAD.cpp 中
#include <pybind11/pybind11.h>

PYBIND11_MODULE("_C", m){}
```

```python
# 例如，在 extension/__init__.py 中
from . import _C
```

2. 本教程中，由于我们重视能够跨多个 CPython 版本构建单个 wheel，因此我们将用稳定的 API 调用替换不稳定的 `PYBIND11` 调用。以下代码使用 `-DPy_LIMITED_API=0x03090000` 进行编译，并成功为我们的 `_C` 扩展创建了一个虚拟 Python 模块，以便可以从 Python 导入它。有关更多详细信息，请参阅 [extension_cpp/__init__.py](https://github.com/pytorch/extension-cpp/blob/38ec45e/extension_cpp/__init__.py) 和 [extension_cpp/csrc/muladd.cpp](https://github.com/pytorch/extension-cpp/blob/38ec45e/extension_cpp/csrc/muladd.cpp)：

```c++
extern "C" {
  PyObject* PyInit__C(void) {
    /**
     * 创建一个可以从 Python 导入的虚拟空 _C 模块。
     * 从 Python 导入将加载此扩展中由此文件组成的 .so，
     * 以便运行下面的 TORCH_LIBRARY 静态初始化程序。
     */
    static struct PyModuleDef module_def = {
      PyModuleDef_HEAD_INIT,
      "_C",   // 模块名称
      NULL,   // 模块文档
      -1,     // 模块每个解释器状态的大小，或者如果模块将状态保存在全局变量中，则为 -1。
      NULL,
    };
    return PyModule_Create(&module_def);
  }
}
```

```python
# 例如，在 extension/__init__.py 中
from . import _C
```

3. 如果您想在 C++ 自定义算子中完全避免使用 `Python.h`，您可以在 Python 中使用 `torch.ops.load_library("/path/to/library.so")` 来加载从扩展编译的 `.so` 文件。请注意，使用此方法时，没有为扩展创建 `_C` Python 模块，因此您无法从 Python 调用 `import _C`。您无需依赖 `import` 语句来触发自定义算子的注册，而是可以使用 `torch.ops.load_library("/path/to/library.so")` 来完成此操作。然后，挑战就转向了解 `.so` 文件的位置，以便您可以加载它们，这并不总是那么简单：

```python
import torch
from pathlib import Path

so_files = list(Path(__file__).parent.glob("_C*.so"))
assert (
    len(so_files) == 1
), f"Expected one _C*.so file, found {len(so_files)}"
torch.ops.load_library(so_files[0])

from . import ops
```

### 为算子添加训练(自动微分)支持

使用 `torch.library.register_autograd` 为算子添加训练支持。最好不要直接使用 Python `torch.autograd.Function` 或 C++ `torch::autograd::Function`；您必须以非常具体的方式使用这些功能，以避免出现无声错误（有关更多详细信息，请参阅[《自定义算子手册》](https://pytorch.org/docs/main/notes/custom_operators.html)）。

In [ ]:
def _backward(ctx, grad):
    a, b = ctx.saved_tensors
    grad_a, grad_b = None, None
    if ctx.needs_input_grad[0]:
        grad_a = grad * b
    if ctx.needs_input_grad[1]:
        grad_b = grad * a
    return grad_a, grad_b, None

def _setup_context(ctx, inputs, output):
    a, b, c = inputs
    saved_a, saved_b = None, None
    if ctx.needs_input_grad[0]:
        saved_b = b
    if ctx.needs_input_grad[1]:
        saved_a = a
    ctx.save_for_backward(saved_a, saved_b)

# 此代码为算子添加了训练支持。您必须向我们提供
# 算子的反向公式和一个 `setup_context` 函数
# 以保存要在反向中使用的值。
torch.library.register_autograd(
    "dengww::mymuladd", _backward, setup_context=_setup_context
)

请注意，反向传播必须是 PyTorch 可理解的算子的组合。如果您希望在反向传播中使用另一个自定义 C++ 或 CUDA 内核，则必须将其包装到自定义算子中。

如果我们有自己的自定义 `mymul` 内核，我们需要将其包装到自定义算子中，然后从反向传播中调用它：

```c++
// New! a mymul_cpu kernel
at::Tensor mymul_cpu(const at::Tensor& a, const at::Tensor& b) {
  TORCH_CHECK(a.sizes() == b.sizes());
  TORCH_CHECK(a.dtype() == at::kFloat);
  TORCH_CHECK(b.dtype() == at::kFloat);
  TORCH_CHECK(a.device().type() == at::DeviceType::CPU);
  TORCH_CHECK(b.device().type() == at::DeviceType::CPU);
  at::Tensor a_contig = a.contiguous();
  at::Tensor b_contig = b.contiguous();
  at::Tensor result = torch::empty(a_contig.sizes(), a_contig.options());
  const float* a_ptr = a_contig.data_ptr<float>();
  const float* b_ptr = b_contig.data_ptr<float>();
  float* result_ptr = result.data_ptr<float>();
  for (int64_t i = 0; i < result.numel(); i++) {
    result_ptr[i] = a_ptr[i] * b_ptr[i];
  }
  return result;
}

TORCH_LIBRARY(extension_cpp, m) {
  m.def("mymuladd(Tensor a, Tensor b, float c) -> Tensor");
  // New! defining the mymul operator
  m.def("mymul(Tensor a, Tensor b) -> Tensor");
}


TORCH_LIBRARY_IMPL(extension_cpp, CPU, m) {
  m.impl("mymuladd", &mymuladd_cpu);
  // New! registering the cpu kernel for the mymul operator
  m.impl("mymul", &mymul_cpu);
}
```

```python
def _backward(ctx, grad):
    a, b = ctx.saved_tensors
    grad_a, grad_b = None, None
    if ctx.needs_input_grad[0]:
        grad_a = torch.ops.extension_cpp.mymul.default(grad, b)
    if ctx.needs_input_grad[1]:
        grad_b = torch.ops.extension_cpp.mymul.default(grad, a)
    return grad_a, grad_b, None


def _setup_context(ctx, inputs, output):
    a, b, c = inputs
    saved_a, saved_b = None, None
    if ctx.needs_input_grad[0]:
        saved_b = b
    if ctx.needs_input_grad[1]:
        saved_a = a
    ctx.save_for_backward(saved_a, saved_b)


# 此代码为算子添加了训练支持。您必须向我们提供
# 算子的反向公式和一个 `setup_context` 函数
# 以保存要在后向中使用的值。
torch.library.register_autograd(
    "extension_cpp::mymuladd", _backward, setup_context=_setup_context)
```

## 测试算子

使用 `torch.library.opcheck` 测试自定义操作是否已正确注册。请注意，此函数不会测试梯度在数学上是否正确 - 计划为此编写单独的测试，无论是手动测试还是使用 `torch.autograd.gradcheck`。

```python
def sample_inputs(device, *, requires_grad=False):
    def make_tensor(*size):
        return torch.randn(size, device=device, requires_grad=requires_grad)

    def make_nondiff_tensor(*size):
        return torch.randn(size, device=device, requires_grad=False)

    return [
        [make_tensor(3), make_tensor(3), 1],
        [make_tensor(20), make_tensor(20), 3.14],
        [make_tensor(20), make_nondiff_tensor(20), -123],
        [make_nondiff_tensor(2, 3), make_tensor(2, 3), -0.3],
    ]

def reference_muladd(a, b, c):
    return a * b + c

samples = sample_inputs(device, requires_grad=True)
samples.extend(sample_inputs(device, requires_grad=False))
for args in samples:
    # Correctness test
    result = torch.ops.extension_cpp.mymuladd(*args)
    expected = reference_muladd(*args)
    torch.testing.assert_close(result, expected)

    # 使用 opcheck 检查操作员注册 API 的错误使用
    torch.library.opcheck(torch.ops.extension_cpp.mymuladd.default, args)
```

## 创建可变算子

您可能希望编写一个自定义算子来改变其输入。使用 `Tensor(a!)` 指定参数中的每个可变 `Tensor`；否则，将出现未定义的行为。如果有多个可变 `Tensor`，请为每个可变 Tensor 使用不同的名称（例如，`Tensor(a!)`、`Tensor(b!)`、`Tensor(c!)`）。

让我们编写一个 `myadd_out(a, b, out)` 算子，它将 `a+b` 的内容写入 `out`。

```c++
// An example of an operator that mutates one of its inputs.
void myadd_out_cpu(const at::Tensor& a, const at::Tensor& b, at::Tensor& out) {
  TORCH_CHECK(a.sizes() == b.sizes());
  TORCH_CHECK(b.sizes() == out.sizes());
  TORCH_CHECK(a.dtype() == at::kFloat);
  TORCH_CHECK(b.dtype() == at::kFloat);
  TORCH_CHECK(out.dtype() == at::kFloat);
  TORCH_CHECK(out.is_contiguous());
  TORCH_INTERNAL_ASSERT(a.device().type() == at::DeviceType::CPU);
  TORCH_INTERNAL_ASSERT(b.device().type() == at::DeviceType::CPU);
  TORCH_INTERNAL_ASSERT(out.device().type() == at::DeviceType::CPU);
  at::Tensor a_contig = a.contiguous();
  at::Tensor b_contig = b.contiguous();
  const float* a_ptr = a_contig.data_ptr<float>();
  const float* b_ptr = b_contig.data_ptr<float>();
  float* result_ptr = out.data_ptr<float>();
  for (int64_t i = 0; i < out.numel(); i++) {
    result_ptr[i] = a_ptr[i] + b_ptr[i];
  }
}
```

在定义算子时，我们必须指定它会改变模式中的输出张量：

```c++
TORCH_LIBRARY(extension_cpp, m) {
  m.def("mymuladd(Tensor a, Tensor b, float c) -> Tensor");
  m.def("mymul(Tensor a, Tensor b) -> Tensor");
  // New!
  m.def("myadd_out(Tensor a, Tensor b, Tensor(a!) out) -> ()");
}

TORCH_LIBRARY_IMPL(extension_cpp, CPU, m) {
  m.impl("mymuladd", &mymuladd_cpu);
  m.impl("mymul", &mymul_cpu);
  // New!
  m.impl("myadd_out", &myadd_out_cpu);
}
```

> 不要将任何可变的张量作为算子的输出返回，因为这会导致与 PyTorch 子系统（如 `torch.compile`）不兼容。

## 总结

在本教程中，我们介绍了将自定义 C++ 和 CUDA 运算符与 PyTorch 集成的推荐方法。`TORCH_LIBRARY/torch.library` API 相当低级。有关如何使用 API 的更多信息，请参阅[自定义运算符手册](https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html#the-custom-operators-manual)。